In [8]:
# !pip install nest_asyncio
from logging.handlers import TimedRotatingFileHandler
import logging.config
from psenti import SentimentAnalysis, SentimentConnection, Document
from sklearn import metrics
from datetime import datetime
import socket
import json
import pandas as pd

user_name = socket.gethostname()
host = 'sentiment2.wikiled.com'
port = 80

import nest_asyncio
import asyncio
nest_asyncio.apply(loop=asyncio.get_event_loop())

logger = logging.getLogger('JupyterUI')
logFormatter = logging.Formatter('%(asctime)s - [%(thread)s] [%(threadName)s]- %(name)s - %(levelname)s - %(message)s')
logger.setLevel(logging.DEBUG)

console = logging.StreamHandler()
console.setFormatter(logFormatter)
console.setLevel(logging.DEBUG)

logger.addHandler(console)

connection = SentimentConnection(host=host, port=port, client_id=user_name)

ConnectionError: HTTPConnectionPool(host='sentiment2.wikiled.com', port=80): Max retries exceeded with url: /api/sentiment/version (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fbe40d0a810>: Failed to establish a new connection: [Errno 110] Connection timed out'))

In [ ]:
reviews = ['Huge profit reported.', 'Huge loss reported.', 'This stock is performing poor.', 'I am not going to invest in this stock.', 
           'I will invest in this stock.', 'putting apple record-breaking quarter into context: $aapl URL_URL via @forbestech']
# reviews = ['I love this hello kitty decal! I like that the bow is pink instead of red. Only bad thing is that after putting it on the window theres a few air bubbles, but that most likely my fault. Shipped fast too.', 
#            'This is the worst scenario case.']
SentimentStars = []
TotalSentimentWords = []
TotalWords = []
Emot= []
for review in reviews:
    TotalWords.append(len(list(review.split(' '))))
def process_result(result):    
    SentimentStars.append(result["Stars"])
    count = 0;
    myMap = dict({'Anger' : 0, 'Anticipation' : 0, 'Disgust' : 0, 'Fear' : 0, 'Joy' : 0, 'Sadness' : 0, 'Surprise' : 0, 'Trust': 0})
    for sentence in result['Sentences']:
        for word in sentence['Words']:
            if 'Emotions' in word and word['Emotions'] != ['None']:
                count = count + 1;
                for emotion in word['Emotions']:
                    myMap[emotion] = myMap[emotion] + 1;
    Emot.append(myMap)
                
    TotalSentimentWords.append(count)

In [ ]:
for review in reviews:
    l = []
    l.append(review)
    analysis = SentimentAnalysis(connection, extract_emotions=True)
    analysis.on_message.subscribe(lambda result: process_result(result))
    analysis.detect_sentiment_text(l)

In [ ]:
df = pd.DataFrame({'SentimentStars' : SentimentStars, 'TotalSentimentWords' : TotalSentimentWords, 'TotalWords' : TotalWords})
df1 = pd.DataFrame(Emot)
df = pd.concat([df, df1], sort=False, axis =1)
df

In [ ]:
from selenium import webdriver
from selenium.webdriver.common import keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [ ]:
comment_list = []
browser = webdriver.Chrome('/home/prachi/Downloads/chromedriver_linux64/chromedriver')
ticker_name = input()
browser.maximize_window()
browser.implicitly_wait(10)

for i in range(500):
    comment = browser.find_element_by_xpath(xpath='/html/body/div[3]/div/div/div[3]/div/div/div[1]/div[2]/div/div/div[2]/div[3]/div/div[{}]/div/div/article/div/div[2]/div[2]/div/div/div'.format(i+1)).text
    comment_list.append(comment)
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")

df = pd.DataFrame(comment_list, columns=['Comments'])

tesla_data = df.to_csv('tesla1_comments.csv', index=False)

In [ ]:
df1 = pd.read_csv (r'/home/prachi/PycharmProjects/pythonProject/tesla_comments.csv')

In [ ]:
SentimentStars = []
TotalSentimentWords = []
TotalWords = []
Emot= []
for i in range(500):
    TotalWords.append(len(list(df1['Comments'][i].split(' '))))
def process_result(result):    
    SentimentStars.append(result["Stars"])
    count = 0;
    myMap = dict({'Anger' : 0, 'Anticipation' : 0, 'Disgust' : 0, 'Fear' : 0, 'Joy' : 0, 'Sadness' : 0, 'Surprise' : 0, 'Trust': 0})
    for sentence in result['Sentences']:
        for word in sentence['Words']:
            if 'Emotions' in word and word['Emotions'] != ['None']:
                count = count + 1;
                for emotion in word['Emotions']:
                    myMap[emotion] = myMap[emotion] + 1;
    Emot.append(myMap)
                
    TotalSentimentWords.append(count)

In [ ]:
for i in range(500):
    l = []
    l.append(df1['Comments'][i])
    analysis = SentimentAnalysis(connection, extract_emotions=True)
    analysis.on_message.subscribe(lambda result: process_result(result))
    analysis.detect_sentiment_text(l)

In [ ]:
df2 = pd.DataFrame({'SentimentStars' : SentimentStars, 'TotalSentimentWords' : TotalSentimentWords, 'TotalWords' : TotalWords})
df3 = pd.DataFrame(Emot)
df2 = pd.concat([df1, df2, df3], sort=False, axis =1)

In [ ]:
df2.to_csv(r'/home/prachi/PycharmProjects/pythonProject/tesla_new_results.csv', index = False)